In [66]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')
sub_df = pd.read_csv('submission.csv')

In [67]:
train_df

,policy_id,age,customer_months,policy_bind_date,policy_state,policy_csl,policy_deductable,policy_annual_premium,umbrella_limit,insured_zip,insured_sex,insured_education_level,insured_occupation,insured_hobbies,insured_relationship,capital-gains,capital-loss,incident_date,incident_type,collision_type,incident_severity,authorities_contacted,incident_state,incident_city,incident_hour_of_the_day,number_of_vehicles_involved,property_damage,bodily_injuries,witnesses,police_report_available,total_claim_amount,injury_claim,property_claim,vehicle_claim,auto_make,auto_model,auto_year,fraud
0,122576,37,189,2013-08-21,C,500/1000,1000,1465.71,5000000,455456,FEMALE,Masters,protective-serv,reading,not-in-family,62203,0,2014-12-22,Single Vehicle Collision,Side Collision,Total Loss,Ambulance,S5,Riverwood,21,1,?,0,3,?,54930,6029,5752,44452,Nissan,Maxima,2000,0
1,937713,44,234,1998-01-04,B,250/500,500,821.24,0,591805,MALE,JD,craft-repair,polo,other-relative,31606,0,2015-02-18,Multi-vehicle Collision,Side Collision,Minor Damage,Other,S5,Springfield,4,3,?,2,1,YES,50680,5376,10156,37347,Honda,Civic,1996,0
2,680237,33,23,1996-02-06,B,500/1000,1000,1844.00,0,442490,FEMALE,High School,machine-op-inspct,skydiving,wife,0,-43166,2015-01-18,Single Vehicle Collision,Side Collision,Total Loss,Police,S3,Northbend,0,1,?,2,1,NO,47829,4460,9247,33644,Jeep,Wrangler,2002,0
3,513080,42,210,2008-11-14,A,500/1000,500,1867.29,0,439408,MALE,JD,transport-moving,video-games,own-child,0,-49440,2015-02-02,Multi-vehicle Collision,Front Collision,Major Damage,Fire,S3,Northbend,20,3,YES,2,2,YES,68862,11043,5955,53548,Suburu,Legacy,2003,1
4,192875,29,81,2002-01-08,A,100/300,1000,816.25,0,640575,FEMALE,MD,craft-repair,video-games,own-child,75296,-73689,2015-02-09,Multi-vehicle Collision,Rear Collision,Total Loss,Fire,S2,Northbend,9,3,YES,2,1,YES,59726,5617,10301,41550,Ford,F150,2004,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
695,1008425,37,196,1997-06-29,C,250/500,500,1301.20,0,474615,MALE,JD,tech-support,video-games,wife,47627,0,2015-01-18,Single Vehicle Collision,Front Collision,Major Damage,Ambulance,S5,Columbus,4,1,?,0,3,NO,61433,10436,11432,39745,Nissan,Pathfinder,2011,1
696,770702,43,229,2001-05-29,A,250/500,500,1434.94,8000000,444476,MALE,College,machine-op-inspct,golf,husband,0,-32289,2015-01-13,Multi-vehicle Collision,Rear Collision,Major Damage,Ambulance,S1,Arlington,17,3,NO,0,1,?,68623,6798,14557,50606,Volkswagen,Passat,2013,1
697,755099,35,209,2003-01-11,C,100/300,500,1639.46,0,639608,FEMALE,College,transport-moving,golf,not-in-family,0,-40797,2015-03-05,Multi-vehicle Collision,Rear Collision,Minor Damage,Fire,S2,Riverwood,7,3,NO,2,0,YES,58033,9129,4598,40740,Mercedes,C300,2002,0
698,693804,44,275,2003-07-22,B,500/1000,2000,1042.29,0,432061,FEMALE,Associate,machine-op-inspct,paintball,other-relative,46822,0,2015-01-09,Multi-vehicle Collision,Rear Collision,Major Damage,Ambulance,S5,Northbend,20,3,?,1,0,NO,35253,7359,3464,24677,Audi,A3,2007,1


In [68]:
test_df

,policy_id,age,customer_months,policy_bind_date,policy_state,policy_csl,policy_deductable,policy_annual_premium,umbrella_limit,insured_zip,insured_sex,insured_education_level,insured_occupation,insured_hobbies,insured_relationship,capital-gains,capital-loss,incident_date,incident_type,collision_type,incident_severity,authorities_contacted,incident_state,incident_city,incident_hour_of_the_day,number_of_vehicles_involved,property_damage,bodily_injuries,witnesses,police_report_available,total_claim_amount,injury_claim,property_claim,vehicle_claim,auto_make,auto_model,auto_year
0,681822,60,473,2002-12-17,B,500/1000,1000,1134.96,0,445975,FEMALE,MD,exec-managerial,camping,other-relative,0,-44262,2015-01-31,Multi-vehicle Collision,Rear Collision,Total Loss,Other,S2,Arlington,9,2,?,0,3,?,53253,5212,10251,39503,Saab,95,2006
1,301288,36,173,1994-01-15,B,100/300,1000,916.20,0,469238,FEMALE,Masters,exec-managerial,camping,other-relative,0,-38591,2015-01-04,Multi-vehicle Collision,Rear Collision,Major Damage,Ambulance,S5,Springfield,22,3,?,0,0,NO,69401,8309,8439,50012,Mercedes,ML350,2008
2,212001,36,147,1995-12-19,B,500/1000,1000,1175.74,5000000,595953,FEMALE,MD,adm-clerical,hiking,own-child,56753,0,2015-02-09,Single Vehicle Collision,Side Collision,Total Loss,Police,S6,Northbrook,0,1,NO,2,0,NO,63919,5572,11477,42801,Dodge,Neon,2009
3,797680,24,71,1992-06-20,C,500/1000,500,1472.40,0,613103,FEMALE,High School,armed-forces,yachting,other-relative,0,0,2015-01-15,Multi-vehicle Collision,Rear Collision,Minor Damage,Police,S1,Springfield,23,3,?,0,0,NO,63173,12027,6500,43423,Dodge,RAM,2012
4,789334,39,230,1996-11-28,C,250/500,1000,1159.44,4000000,581581,FEMALE,Masters,exec-managerial,reading,wife,0,-62877,2015-01-08,Parked Car,?,Minor Damage,Police,S1,Springfield,17,1,YES,0,0,?,8847,904,1786,6138,Accura,RSX,2003
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,663065,36,30,1999-08-18,B,500/1000,2000,1384.15,9000000,593323,FEMALE,College,prof-specialty,kayaking,not-in-family,0,-79936,2015-01-14,Parked Car,?,Trivial Damage,None,S3,Arlington,9,1,NO,0,1,YES,4507,970,477,3339,Dodge,Neon,2002
296,283767,47,285,2009-12-23,C,250/500,500,1590.78,7000000,447235,MALE,MD,adm-clerical,movies,unmarried,0,-58786,2015-02-09,Multi-vehicle Collision,Side Collision,Total Loss,Fire,S3,Columbus,0,3,?,0,3,YES,45909,5599,5627,34598,Jeep,Grand Cherokee,1999
297,325099,39,256,1999-04-08,C,500/1000,2000,1265.24,0,592069,FEMALE,Associate,other-service,hiking,not-in-family,0,-31129,2014-12-21,Single Vehicle Collision,Rear Collision,Minor Damage,Police,S1,Hillsdale,10,1,YES,0,0,?,42293,5773,5491,34805,Dodge,RAM,1997
298,465673,35,54,2010-09-08,C,100/300,500,1229.74,0,451451,FEMALE,MD,protective-serv,hiking,unmarried,65965,-43489,2015-01-27,Multi-vehicle Collision,Side Collision,Minor Damage,Fire,S4,Springfield,21,3,YES,2,0,?,76875,14955,7312,59418,Nissan,Maxima,2012


In [69]:
raw_df = pd.concat([train_df, test_df])
raw_df

,policy_id,age,customer_months,policy_bind_date,policy_state,policy_csl,policy_deductable,policy_annual_premium,umbrella_limit,insured_zip,insured_sex,insured_education_level,insured_occupation,insured_hobbies,insured_relationship,capital-gains,capital-loss,incident_date,incident_type,collision_type,incident_severity,authorities_contacted,incident_state,incident_city,incident_hour_of_the_day,number_of_vehicles_involved,property_damage,bodily_injuries,witnesses,police_report_available,total_claim_amount,injury_claim,property_claim,vehicle_claim,auto_make,auto_model,auto_year,fraud
0,122576,37,189,2013-08-21,C,500/1000,1000,1465.71,5000000,455456,FEMALE,Masters,protective-serv,reading,not-in-family,62203,0,2014-12-22,Single Vehicle Collision,Side Collision,Total Loss,Ambulance,S5,Riverwood,21,1,?,0,3,?,54930,6029,5752,44452,Nissan,Maxima,2000,0.0
1,937713,44,234,1998-01-04,B,250/500,500,821.24,0,591805,MALE,JD,craft-repair,polo,other-relative,31606,0,2015-02-18,Multi-vehicle Collision,Side Collision,Minor Damage,Other,S5,Springfield,4,3,?,2,1,YES,50680,5376,10156,37347,Honda,Civic,1996,0.0
2,680237,33,23,1996-02-06,B,500/1000,1000,1844.00,0,442490,FEMALE,High School,machine-op-inspct,skydiving,wife,0,-43166,2015-01-18,Single Vehicle Collision,Side Collision,Total Loss,Police,S3,Northbend,0,1,?,2,1,NO,47829,4460,9247,33644,Jeep,Wrangler,2002,0.0
3,513080,42,210,2008-11-14,A,500/1000,500,1867.29,0,439408,MALE,JD,transport-moving,video-games,own-child,0,-49440,2015-02-02,Multi-vehicle Collision,Front Collision,Major Damage,Fire,S3,Northbend,20,3,YES,2,2,YES,68862,11043,5955,53548,Suburu,Legacy,2003,1.0
4,192875,29,81,2002-01-08,A,100/300,1000,816.25,0,640575,FEMALE,MD,craft-repair,video-games,own-child,75296,-73689,2015-02-09,Multi-vehicle Collision,Rear Collision,Total Loss,Fire,S2,Northbend,9,3,YES,2,1,YES,59726,5617,10301,41550,Ford,F150,2004,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,663065,36,30,1999-08-18,B,500/1000,2000,1384.15,9000000,593323,FEMALE,College,prof-specialty,kayaking,not-in-family,0,-79936,2015-01-14,Parked Car,?,Trivial Damage,None,S3,Arlington,9,1,NO,0,1,YES,4507,970,477,3339,Dodge,Neon,2002,NaN
296,283767,47,285,2009-12-23,C,250/500,500,1590.78,7000000,447235,MALE,MD,adm-clerical,movies,unmarried,0,-58786,2015-02-09,Multi-vehicle Collision,Side Collision,Total Loss,Fire,S3,Columbus,0,3,?,0,3,YES,45909,5599,5627,34598,Jeep,Grand Cherokee,1999,NaN
297,325099,39,256,1999-04-08,C,500/1000,2000,1265.24,0,592069,FEMALE,Associate,other-service,hiking,not-in-family,0,-31129,2014-12-21,Single Vehicle Collision,Rear Collision,Minor Damage,Police,S1,Hillsdale,10,1,YES,0,0,?,42293,5773,5491,34805,Dodge,RAM,1997,NaN
298,465673,35,54,2010-09-08,C,100/300,500,1229.74,0,451451,FEMALE,MD,protective-serv,hiking,unmarried,65965,-43489,2015-01-27,Multi-vehicle Collision,Side Collision,Minor Damage,Fire,S4,Springfield,21,3,YES,2,0,?,76875,14955,7312,59418,Nissan,Maxima,2012,NaN


In [70]:
raw_df['incident_date'] = pd.to_datetime(raw_df['incident_date'], format='%Y-%m-%d')
raw_df

,policy_id,age,customer_months,policy_bind_date,policy_state,policy_csl,policy_deductable,policy_annual_premium,umbrella_limit,insured_zip,insured_sex,insured_education_level,insured_occupation,insured_hobbies,insured_relationship,capital-gains,capital-loss,incident_date,incident_type,collision_type,incident_severity,authorities_contacted,incident_state,incident_city,incident_hour_of_the_day,number_of_vehicles_involved,property_damage,bodily_injuries,witnesses,police_report_available,total_claim_amount,injury_claim,property_claim,vehicle_claim,auto_make,auto_model,auto_year,fraud
0,122576,37,189,2013-08-21,C,500/1000,1000,1465.71,5000000,455456,FEMALE,Masters,protective-serv,reading,not-in-family,62203,0,2014-12-22,Single Vehicle Collision,Side Collision,Total Loss,Ambulance,S5,Riverwood,21,1,?,0,3,?,54930,6029,5752,44452,Nissan,Maxima,2000,0.0
1,937713,44,234,1998-01-04,B,250/500,500,821.24,0,591805,MALE,JD,craft-repair,polo,other-relative,31606,0,2015-02-18,Multi-vehicle Collision,Side Collision,Minor Damage,Other,S5,Springfield,4,3,?,2,1,YES,50680,5376,10156,37347,Honda,Civic,1996,0.0
2,680237,33,23,1996-02-06,B,500/1000,1000,1844.00,0,442490,FEMALE,High School,machine-op-inspct,skydiving,wife,0,-43166,2015-01-18,Single Vehicle Collision,Side Collision,Total Loss,Police,S3,Northbend,0,1,?,2,1,NO,47829,4460,9247,33644,Jeep,Wrangler,2002,0.0
3,513080,42,210,2008-11-14,A,500/1000,500,1867.29,0,439408,MALE,JD,transport-moving,video-games,own-child,0,-49440,2015-02-02,Multi-vehicle Collision,Front Collision,Major Damage,Fire,S3,Northbend,20,3,YES,2,2,YES,68862,11043,5955,53548,Suburu,Legacy,2003,1.0
4,192875,29,81,2002-01-08,A,100/300,1000,816.25,0,640575,FEMALE,MD,craft-repair,video-games,own-child,75296,-73689,2015-02-09,Multi-vehicle Collision,Rear Collision,Total Loss,Fire,S2,Northbend,9,3,YES,2,1,YES,59726,5617,10301,41550,Ford,F150,2004,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,663065,36,30,1999-08-18,B,500/1000,2000,1384.15,9000000,593323,FEMALE,College,prof-specialty,kayaking,not-in-family,0,-79936,2015-01-14,Parked Car,?,Trivial Damage,None,S3,Arlington,9,1,NO,0,1,YES,4507,970,477,3339,Dodge,Neon,2002,NaN
296,283767,47,285,2009-12-23,C,250/500,500,1590.78,7000000,447235,MALE,MD,adm-clerical,movies,unmarried,0,-58786,2015-02-09,Multi-vehicle Collision,Side Collision,Total Loss,Fire,S3,Columbus,0,3,?,0,3,YES,45909,5599,5627,34598,Jeep,Grand Cherokee,1999,NaN
297,325099,39,256,1999-04-08,C,500/1000,2000,1265.24,0,592069,FEMALE,Associate,other-service,hiking,not-in-family,0,-31129,2014-12-21,Single Vehicle Collision,Rear Collision,Minor Damage,Police,S1,Hillsdale,10,1,YES,0,0,?,42293,5773,5491,34805,Dodge,RAM,1997,NaN
298,465673,35,54,2010-09-08,C,100/300,500,1229.74,0,451451,FEMALE,MD,protective-serv,hiking,unmarried,65965,-43489,2015-01-27,Multi-vehicle Collision,Side Collision,Minor Damage,Fire,S4,Springfield,21,3,YES,2,0,?,76875,14955,7312,59418,Nissan,Maxima,2012,NaN


In [71]:
raw_df.columns

Index(['policy_id', 'age', 'customer_months', 'policy_bind_date',
       'policy_state', 'policy_csl', 'policy_deductable',
       'policy_annual_premium', 'umbrella_limit', 'insured_zip', 'insured_sex',
       'insured_education_level', 'insured_occupation', 'insured_hobbies',
       'insured_relationship', 'capital-gains', 'capital-loss',
       'incident_date', 'incident_type', 'collision_type', 'incident_severity',
       'authorities_contacted', 'incident_state', 'incident_city',
       'incident_hour_of_the_day', 'number_of_vehicles_involved',
       'property_damage', 'bodily_injuries', 'witnesses',
       'police_report_available', 'total_claim_amount', 'injury_claim',
       'property_claim', 'vehicle_claim', 'auto_make', 'auto_model',
       'auto_year', 'fraud'],
      dtype='object')

In [72]:
raw_df['incident_date']

0     2014-12-22
1     2015-02-18
2     2015-01-18
3     2015-02-02
4     2015-02-09
         ...    
295   2015-01-14
296   2015-02-09
297   2014-12-21
298   2015-01-27
299   2015-01-29
Name: incident_date, Length: 1000, dtype: datetime64[ns]

In [73]:
import datetime

startDate = datetime.datetime.strptime('2022-10-13', '%Y-%m-%d')
raw_df['time'] = raw_df['incident_date'].apply(lambda x: startDate-x).dt.days

raw_df

,policy_id,age,customer_months,policy_bind_date,policy_state,policy_csl,policy_deductable,policy_annual_premium,umbrella_limit,insured_zip,insured_sex,insured_education_level,insured_occupation,insured_hobbies,insured_relationship,capital-gains,capital-loss,incident_date,incident_type,collision_type,incident_severity,authorities_contacted,incident_state,incident_city,incident_hour_of_the_day,number_of_vehicles_involved,property_damage,bodily_injuries,witnesses,police_report_available,total_claim_amount,injury_claim,property_claim,vehicle_claim,auto_make,auto_model,auto_year,fraud,time
0,122576,37,189,2013-08-21,C,500/1000,1000,1465.71,5000000,455456,FEMALE,Masters,protective-serv,reading,not-in-family,62203,0,2014-12-22,Single Vehicle Collision,Side Collision,Total Loss,Ambulance,S5,Riverwood,21,1,?,0,3,?,54930,6029,5752,44452,Nissan,Maxima,2000,0.0,2852
1,937713,44,234,1998-01-04,B,250/500,500,821.24,0,591805,MALE,JD,craft-repair,polo,other-relative,31606,0,2015-02-18,Multi-vehicle Collision,Side Collision,Minor Damage,Other,S5,Springfield,4,3,?,2,1,YES,50680,5376,10156,37347,Honda,Civic,1996,0.0,2794
2,680237,33,23,1996-02-06,B,500/1000,1000,1844.00,0,442490,FEMALE,High School,machine-op-inspct,skydiving,wife,0,-43166,2015-01-18,Single Vehicle Collision,Side Collision,Total Loss,Police,S3,Northbend,0,1,?,2,1,NO,47829,4460,9247,33644,Jeep,Wrangler,2002,0.0,2825
3,513080,42,210,2008-11-14,A,500/1000,500,1867.29,0,439408,MALE,JD,transport-moving,video-games,own-child,0,-49440,2015-02-02,Multi-vehicle Collision,Front Collision,Major Damage,Fire,S3,Northbend,20,3,YES,2,2,YES,68862,11043,5955,53548,Suburu,Legacy,2003,1.0,2810
4,192875,29,81,2002-01-08,A,100/300,1000,816.25,0,640575,FEMALE,MD,craft-repair,video-games,own-child,75296,-73689,2015-02-09,Multi-vehicle Collision,Rear Collision,Total Loss,Fire,S2,Northbend,9,3,YES,2,1,YES,59726,5617,10301,41550,Ford,F150,2004,0.0,2803
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,663065,36,30,1999-08-18,B,500/1000,2000,1384.15,9000000,593323,FEMALE,College,prof-specialty,kayaking,not-in-family,0,-79936,2015-01-14,Parked Car,?,Trivial Damage,None,S3,Arlington,9,1,NO,0,1,YES,4507,970,477,3339,Dodge,Neon,2002,NaN,2829
296,283767,47,285,2009-12-23,C,250/500,500,1590.78,7000000,447235,MALE,MD,adm-clerical,movies,unmarried,0,-58786,2015-02-09,Multi-vehicle Collision,Side Collision,Total Loss,Fire,S3,Columbus,0,3,?,0,3,YES,45909,5599,5627,34598,Jeep,Grand Cherokee,1999,NaN,2803
297,325099,39,256,1999-04-08,C,500/1000,2000,1265.24,0,592069,FEMALE,Associate,other-service,hiking,not-in-family,0,-31129,2014-12-21,Single Vehicle Collision,Rear Collision,Minor Damage,Police,S1,Hillsdale,10,1,YES,0,0,?,42293,5773,5491,34805,Dodge,RAM,1997,NaN,2853
298,465673,35,54,2010-09-08,C,100/300,500,1229.74,0,451451,FEMALE,MD,protective-serv,hiking,unmarried,65965,-43489,2015-01-27,Multi-vehicle Collision,Side Collision,Minor Damage,Fire,S4,Springfield,21,3,YES,2,0,?,76875,14955,7312,59418,Nissan,Maxima,2012,NaN,2816


In [74]:
# Encoder
categorical_cols = raw_df.select_dtypes(include=['object']).columns.tolist()
categorical_cols

['policy_bind_date',
 'policy_state',
 'policy_csl',
 'insured_sex',
 'insured_education_level',
 'insured_occupation',
 'insured_hobbies',
 'insured_relationship',
 'incident_type',
 'collision_type',
 'incident_severity',
 'authorities_contacted',
 'incident_state',
 'incident_city',
 'property_damage',
 'police_report_available',
 'auto_make',
 'auto_model']

In [75]:
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()
for col in categorical_cols:
    encoder.fit(raw_df[col].values)
    raw_df[col] = encoder.transform(raw_df[col].values)

print("Data preprocess is done.")

Data preprocess is done.


In [76]:
testA = raw_df[raw_df['fraud'].isnull()].drop(['policy_id', 'incident_date', 'fraud'], axis=1)
trainA = raw_df[raw_df['fraud'].notnull()]

data_x = trainA.drop(['policy_id', 'incident_date', 'fraud'], axis=1)
data_y = train_df[['fraud']].copy()

col = ['policy_state', 'insured_sex', 'insured_education_level', 'incident_type', 'collision_type', 'incident_severity',
'authorities_contacted', 'incident_state', 'incident_city', 'police_report_available', 'auto_make', 'auto_model']

In [77]:
for i in data_x.columns:
    if i in col:
        data_x[i] = data_x[i].astype('str')

for i in testA.columns:
    if i in col:
        testA[i] = testA[i].astype('str')

In [78]:
from catboost import CatBoostClassifier

model = CatBoostClassifier(
    loss_function='Logloss',
    eval_metric='AUC',
    task_type='CPU',
    learning_rate=0.1,
    iterations=10000,
    random_seed=2020,
    od_type='Iter',
    depth=7,
    early_stopping_rounds=300
)

In [79]:
answer = []
mean_score = 0
n_folds = 10

In [80]:
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score

sk = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=2019)
for train, test in sk.split(data_x, data_y):
    x_train = data_x.iloc[train]
    y_train = data_y.iloc[train]
    x_test = data_x.iloc[test]
    y_test = data_y.iloc[test]

    clf = model.fit(x_train,y_train, eval_set=(x_test,y_test),verbose=500,cat_features=col)
    yy_pred_valid=clf.predict(x_test)
    print('AUC from cat:{}'.format(roc_auc_score(y_test, yy_pred_valid)))
    
    mean_score += roc_auc_score(y_test, yy_pred_valid) / n_folds
    y_pred_valid = clf.predict(testA,prediction_type='Probability')[:,-1]
    answer.append(y_pred_valid)

print('10 folds average AUC:{}'.format(mean_score))

0:	test: 0.7564103	best: 0.7564103 (0)	total: 8.41ms	remaining: 1m 24s
Stopped by overfitting detector  (300 iterations wait)

bestTest = 0.8563034188
bestIteration = 2

Shrink model to first 3 iterations.
AUC from cat:0.7008547008547008
0:	test: 0.6997863	best: 0.6997863 (0)	total: 11.5ms	remaining: 1m 55s
500:	test: 0.8514957	best: 0.8696581 (266)	total: 25.8s	remaining: 8m 9s
Stopped by overfitting detector  (300 iterations wait)

bestTest = 0.8696581197
bestIteration = 266

Shrink model to first 267 iterations.
AUC from cat:0.764957264957265
0:	test: 0.8782051	best: 0.8782051 (0)	total: 8.91ms	remaining: 1m 29s
Stopped by overfitting detector  (300 iterations wait)

bestTest = 0.9476495726
bestIteration = 29

Shrink model to first 30 iterations.
AUC from cat:0.8226495726495727
0:	test: 0.7467949	best: 0.7467949 (0)	total: 8.76ms	remaining: 1m 27s
Stopped by overfitting detector  (300 iterations wait)

bestTest = 0.8397435897
bestIteration = 5

Shrink model to first 6 iterations.
AU

In [81]:
lgb_pre = sum(answer) / n_folds
sub_df['fraud'] = lgb_pre
sub_df.to_csv('fin2predict.csv', index=True)